In [ ]:
"""
The Function that will Populate a 
table with statistics of from different null graphs
Defined within the range of node/edge combinations 
predetermined by Preparing_Node_Edge_Restrictions.ipynb

Pseudocode:
1) Load in the restrictions going to run
2) Create a restriction string that
ORs all the specific restrictions together
--> creates new restriced table
3) Create a computed table that pulls from 
NullModelParamaters_restricted_table:
a. Pull down the model paramater
b. For n_graph_iterations:
    i. Creates the graph with those specified parameters
    ii. Runs the certain statistics on it
    iii. Writes the statistics to the database

"""

In [ ]:
import numpy as np
import graph_statistics_and_simulations as g_stat
import graph_generating_functions_library as g_gen
import time

# Loading in and Creating the Restriction


In [ ]:
n_e_restrictions_data = np.load("Viable_node_edge_restrictions.npz",allow_pickle=True)
n_e_restrictions = n_e_restrictions_data["saved_node_edge_windows"]
ex_restriction = n_e_restrictions[0]

In [ ]:
""" Costructing the large restriction """
individual_restrictions = ["(" + " AND ".join([
    f"n={j['n_nodes']}",
    f"edge_average>={j['edge_min']}",
    f"edge_average<={j['edge_max']}" + " )"
]) for j in n_e_restrictions]

total_restriction = " OR ".join(individual_restrictions)
total_restriction

In [ ]:
"""
Create restricted data table
"""

import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
dj.config["display.limit"] = 20
dj.config['enable_python_native_blobs'] = True

In [ ]:
restricted_null_models = m65.NullModelParameters & total_restriction
len(m65.NullModelParameters()),len(restricted_null_models)

# Figuring Out what statistics to run and function to run them all
# (Will just store them as blobs)

In [ ]:
list_of_functions=[k for k in dir(g_stat) if "directional" in dir(getattr(g_stat,k))]
list_function_not_run = ['smallest_laplacian_eigen_value',
                        'node_connectivity',
                        "average_degree_connectivity",
                        "random_degree_site_percolation",
                        "pandemic_beta",
                        "power_law_alpha_sigma"]

for fn in list_function_not_run:
    list_of_functions.remove(fn)
list_of_functions = np.sort(list_of_functions)

functions_list = [getattr(g_stat,k) for k in list_of_functions]
functions_list

In [ ]:
def run_single_graph_experiment(graph_function,
                               functions_list,
                               graph_parameters,
                               print_flag=False,
                               attr=[]):
    global_start = time.time()
    
    #gets the names of all of the stats going to run
    stats_list = [k.__name__ for k in functions_list]
    
    total_stats_list = dict([(k,[]) for k in stats_list])
    if print_flag:
        print(f"graph_function = {graph_function}")
    graph_type = graph_function.__name__
    
    
    data_structure = dict()
    
    loop_start = time.time()
    graph_args = graph_parameters
    
    #creating the graph with randomize locations
    creation_time = time.time()
    max_graph_creation_tries = 10
    creation_flag = False
    for i in range(0,max_graph_creation_tries):
        try:
            if print_flag:
                print("graph_args = " + str(graph_parameters))
            G = graph_function(**graph_parameters)
            creation_flag = True
            break
        except:
            if print_flag:
                print(f"Graph creationg Failed, starting try #{i + 1}")

    if not creation_flag:
        raise Exception(f"Couldn't build graph after {max_graph_creation_tries} tries")


    if print_flag:
        print(f"   Graph Creation time = {time.time() - creation_time}")

    full_graph_name = str(graph_type) + ":"
    for k,v in graph_parameters.items():
        if type(v) == float:
            full_graph_name += f"_{k}={v:.2f}"
        else:
            full_graph_name += f"_{k}={v}"
    for at in attr:
        full_graph_name += "_" + str(at)
        
    
    data_structure = dict()
    data_structure["graph_name"] = full_graph_name
    data_structure["graph_type"] = graph_type
    data_structure["n_nodes"] = len(G.nodes)
    data_structure["n_edges"] = len(G.edges())
    for k,v in graph_parameters.items():
        data_structure[k] = v
        
    stats_dict_update = g_stat.graph_statistics_vp2(G,functions_list,
                                             undirected_flag = False,
                                             time_print_flag=print_flag,
                                             output_print_flag = print_flag)
    data_structure.update(stats_dict_update)
    
    if print_flag:
        print(f"    Total time for iteration = {time.time() - global_start}")
        
    return data_structure

# Practice Running Statistics on One Graph

In [ ]:
example_graph = (m65.NullModelParameters & "graph_hash=94759").fetch1()
example_graph

#getting the right graph function 
total_functions_modules = [z for z in dir(g_gen) if 
                           type(getattr(g_gen,z)) == type(np.sum)]
matching_function = [getattr(g_gen,j) for j in total_functions_modules if 
                               getattr(g_gen,j).__name__ == example_graph["graph_name"]]

if len(matching_function) != 1:
    print("Number of matching functions is not exactly one")
else:
    g_func = matching_function[0]
    
#getting the right parameters
param_exclusion_list = ["graph_hash","graph_name","edge_average","edge_deviation"]
param_dict = dict([(k,v) for k,v in example_graph.items() if 
                  ((v != None and v != np.nan) and
                  (k not in param_exclusion_list))])
param_dict

In [ ]:
# n_iterations = 2
# start_time = time.time()
# total_insert_dict = []
# for i in range(0,n_iterations):
#     returned_dict = run_single_graph_experiment(graph_function=g_func,
#                                functions_list=functions_list,
#                                graph_parameters=param_dict,
#                                print_flag=True)
#     insert_dict = dict(graph_name=returned_dict["graph_name"],
#                    run_number = i,
#                   graph_type =returned_dict["graph_type"],
#                   n_nodes=returned_dict["n_nodes"],
#                   n_edges=returned_dict["n_edges"],
#                   statistics=returned_dict)
#     total_insert_dict.append(insert_dict)
    
# total_insert_dict
# print(f"Total time = {time.time() - start_time}")

# Creating the Table

In [ ]:
#RandomNetworkStats.delete()

In [ ]:
# my_stat = (RandomNetworkStats() & "graph_hash=1577292" & "run_number=0").fetch1("statistics")
# my_stat

In [ ]:
n_iterations = 50
@schema
class RandomNetworkStats(dj.Computed):
    definition="""
    -> m65.NullModelParameters
    graph_name: char(120)  # the name of the run with all of the parameters
    run_number: int unsigned #the number run for this graph
    ---
    graph_type : char(60)  # The exact type of graph run
    n_nodes:          int unsigned      # number of nodes
    n_edges:          int unsigned      # number of edges in graph
    statistics:       longblob  #stores all of the statistics in the run
    """

    key_source = m65.NullModelParameters & total_restriction
    
    def make(self,key):
        
        global_time = time.time()
        example_graph = (m65.NullModelParameters & key).fetch1()
        print(f"\n\n -- Working on {example_graph} -- ")

        #getting the right graph function 
        total_functions_modules = [z for z in dir(g_gen) if 
                                   type(getattr(g_gen,z)) == type(np.sum)]
        matching_function = [getattr(g_gen,j) for j in total_functions_modules if 
                                       getattr(g_gen,j).__name__ == example_graph["graph_name"]]

        if len(matching_function) != 1:
            raise Exception("Number of matching functions is not exactly one")
            
        else:
            g_func = matching_function[0]

        #getting the right parameters
        param_exclusion_list = ["graph_hash","graph_name","edge_average","edge_deviation"]
        param_dict = dict([(k,v) for k,v in example_graph.items() if 
                          ((v != None and v != np.nan) and
                          (k not in param_exclusion_list))])
        
        
        start_time = time.time()
        total_insert_dict = []
        for i in range(0,n_iterations):
            returned_dict = run_single_graph_experiment(graph_function=g_func,
                                       functions_list=functions_list,
                                       graph_parameters=param_dict,
                                       print_flag=False)
            insert_dict = dict(graph_hash=key["graph_hash"],
                graph_name=returned_dict["graph_name"],
                           run_number = i,
                          graph_type =returned_dict["graph_type"],
                          n_nodes=returned_dict["n_nodes"],
                          n_edges=returned_dict["n_edges"],
                          statistics=returned_dict)
            total_insert_dict.append(insert_dict)
        
        print(f"Total time for {n_iterations} iterations: {time.time() - global_time}")
        #now do the insert
        self.insert(total_insert_dict,skip_duplicates=True)
        

In [ ]:
#(schema.jobs & "table_name='__random_network_stats'").delete()

In [ ]:
start_time = time.time()
RandomNetworkStats.populate(reserve_jobs=True)
print(f"Total time for RandomNetworkStats populate = {time.time() - start_time}")